In [1]:
from langchain.document_loaders import WebBaseLoader

# List of URLs for stock data
urls = [
    "https://finance.yahoo.com/quote/AAPL",  # Yahoo Finance - Apple
    "https://www.investing.com/equities/apple-computer-inc",  # Investing.com - Apple
    "https://www.moneycontrol.com/india/stockpricequote/technology/infosys/IT"  # Moneycontrol - Infosys
]

# Load documents from all URLs
loader = WebBaseLoader(urls)
docs = loader.load()


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter()

# Split documents into chunks
doc_chunks = text_splitter.split_documents(docs)
print(f"Number of document chunks: {len(doc_chunks)}")


Number of document chunks: 41


In [2]:
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain import LLMChain, PromptTemplate
import os
import tempfile
import shutil
import sys